In [ ]:
import pandas as pd
import numpy as np

import xarray as xr # -> para funcionar é necessario instalar de forma complementar -> pandas -m pip install netCDF4 h5netcdf scipy pydap zarr fsspec cftime rasterio cfgrib pooch 

import rioxarray as rio
import geopandas as gpd

import fiona

import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, MultiPolygon


In [ ]:
# FUNÇÃO PARA CONVERTER (e salvar) ARQUIVOS .NC4 EM SHAPEFILES 
def convertion(nc4_list,export_name): 
  data_frames = []
  for obj in nc4_list:
    nc_file = xr.open_dataset(obj['address'])

    xco2 = nc_file[[
      'longitude',
      'latitude',
      'xco2',
      'time',
      'vertex_longitude',
      'vertex_latitude'
    ]]

    xco2.rio.set_crs("epsg:4326")

    df = xco2.to_dataframe()

    df = df.reset_index()

    df['vertices'] = df['vertices'].astype(str)

    dfp = pd.pivot_table(df,
      index=['sounding_id', 'time', 'latitude', 'longitude', 'xco2'],
      columns='vertices',
      values=['vertex_longitude', 'vertex_latitude']
    )

    dfp = dfp.reset_index()

    dfp.columns = dfp.columns.map('_'.join).str.strip('_')

    dfp = dfp.rename(columns={'sounding_id': 'index'})
    # dfp.Index()

    dfp['time'] = dfp['time'].astype(str)
    dfp['year'] = obj['year']
    dfp['month'] = obj['month']
    dfp['day'] = dfp['time'][0].split(' ')[0].split('-')[2]
    dfp['id_shape'] = nc4_list.index(obj)

    dfp['geometry'] = dfp.apply(
        lambda s: Polygon(shell=[
            Point(s.vertex_longitude_1, s.vertex_latitude_1),
            Point(s.vertex_longitude_2, s.vertex_latitude_2),
            Point(s.vertex_longitude_3, s.vertex_latitude_3),
            Point(s.vertex_longitude_4, s.vertex_latitude_4),
            Point(s.vertex_longitude_1, s.vertex_latitude_1)
        ]),
        axis=1
    )
  
    dfp.pop('vertex_longitude_1')
    dfp.pop('vertex_longitude_2')
    dfp.pop('vertex_longitude_3')
    dfp.pop('vertex_longitude_4')
    dfp.pop('vertex_latitude_1')
    dfp.pop('vertex_latitude_2')
    dfp.pop('vertex_latitude_3')
    dfp.pop('vertex_latitude_4')
    dfp.pop('longitude')
    dfp.pop('latitude')
  
    gdf = gpd.GeoDataFrame(dfp, geometry="geometry", crs="EPSG:4326")
    
    gdf['area'] = gdf['geometry'].area/ 10**6
    address = 'shps-v1/{0}-{1}-{2}-v0.shp'.format(export_name,dfp['time'][0].split(' ')[0].split('-')[2],nc4_list.index(obj))

    gdf.to_file(address)  
  

In [ ]:
address = 'C:/Users/wallace.silva/Desktop/workspace/mapbiomas-air/data/nc4s-v1'
nc4_files = os.listdir(address)

len(nc4_files)

2066

In [ ]:
#  ORGANIZANDO A LISTA DE STRINGS EM LISTA OBJETOS
month_to_number = {
    'janeiro':'01',
    'fevereiro':'02',
    'marco':'03',
    'abril':'04',
    'maio':'05',
    'junho':'06',
    'julho':'07',
    'agosto':'08',
    'setembro':'09',
    'outubro':'10',
    'novembro':'11',
    'dezembro':'12',
}

obj_list = []

for file in nc4_files:
    split = file.split('_')
    month_str = split[2].replace('-oco2','')
    obj = {
        'year':split[1],
        'month_str':month_str,
        'month':month_to_number[month_str],
        'address':address + '/' + file
    }

    obj_list.append(obj)



In [32]:
# ESQUEMATIZANDO OS LOOPS PARA AGREGAR SHAPEFILES MENSAIS
# lista com todos os anos 
years = [obj['year'] for obj in obj_list]
years =  sorted(list(set(years)))
years = [
'2020',
# '2021'
]
# print(type(years),years)
# years = years[3:5]
print(years)
print('start process')
for year in years:
    year_filter = lambda x: x["year"] == year
    year_list = list(filter(year_filter,obj_list))
    
    months = [obj['month'] for obj in year_list]
    months = sorted(list(set(months)))
    months = ['05','06','07','08','09','10','11','12']
    for month in months:
        print('INFO: {0}-{1}   ->   years: {2}/{3}    months: {4}/{5}'.format(year,month,years.index(year) + 1,len(years),months.index(month) + 1,len(months)))
        month_filter = lambda x: x["month"] == month
        month_list = list(filter(month_filter,year_list))
        convertion(month_list,'oco2-ppm-{0}-{1}'.format(year,month))
print('concluido')

['2020']
start process
INFO: 2020-05   ->   years: 1/1    months: 1/8
